# Download yolov5 repository

In [2]:
!git clone https://github.com/ultralytics/yolov5.git

fatal: destination path 'yolov5' already exists and is not an empty directory.


# download weights

In [3]:
!wget https://doc-0s-8o-docs.googleusercontent.com/docs/securesc/qkaugo6314u8fv5lmscc33kqv5o3bopg/t810lfv1p8l94jf463k5rkpqjd2crpib/1646082975000/07750003106284155945/16543569848639810509/1W4E7kawGQfTV2k0LxCDPCnPHc32yj6pv?e=download

--2022-03-02 23:05:13--  https://doc-0s-8o-docs.googleusercontent.com/docs/securesc/qkaugo6314u8fv5lmscc33kqv5o3bopg/t810lfv1p8l94jf463k5rkpqjd2crpib/1646082975000/07750003106284155945/16543569848639810509/1W4E7kawGQfTV2k0LxCDPCnPHc32yj6pv?e=download
Resolving doc-0s-8o-docs.googleusercontent.com (doc-0s-8o-docs.googleusercontent.com)... 142.250.187.193, 2a00:1450:4009:81f::2001
Connecting to doc-0s-8o-docs.googleusercontent.com (doc-0s-8o-docs.googleusercontent.com)|142.250.187.193|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2022-03-02 23:05:14 ERROR 403: Forbidden.



# Pipeline todo
- get video from vantage platform (or wget a pre-downloaded video for POC)
- split into frames  or load video frame per frame using open cv
- run yolo on each frame recording bbox of each image in yolo format
- convert yolo format bbox to geographic format using frame metadata and projection
    - display on map for POC
- create binary image which is geo based
- run particle filter or something similar using CV2 to track objects
- use tracked objects positioning etc. to calculate speed and heading
- display this both in a map and on the original video

In [ ]:
#Example reading per frame
import cv2
import torch

video = cv2.VideoCapture('input/test.mp4')
model = torch.hub.load('input/yolov5', 'yolov5s') 
while True:
    ret, frame = video.read()
    if not ret:
        break
    
    results = model(frame)
    results.print()  # or .show(), .save(), .crop(), .pandas(), etc.
    
video.release()

In [ ]:
#example to search for airport by IATACode and download video
import getpass
from vantage_api.services import VantageServiceApi
from vantage_api.utils import GeoRequest
from airport_database.airports import Airports

user = getpass.getpass('Username: ')
apiKey = getpass.getpass('API Key: ')

vantageApi = VantageServiceApi(user, apiKey)

nzAirport = Airports().findByIATACode('AKL')

geoData = GeoRequest()
geoData.fromBoundingBox(nzAirport.boundingBox(halfSideInKm=10))

features = vantageApi.getSearch(geoData).getFeatures()
keys = list(features.keys())
firstFeature = features[keys[0]]

outputFiles = vantageApi.downloadFiles(firstFeature,'./input', verbose=True)
